<a href="https://colab.research.google.com/github/Isabela-SAG/Isabela-SAG/blob/main/datathon_linkedin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install linkdapi

In [6]:
import json

file_path = '/content/linkedin_success_master.json'

try:
    with open(file_path, 'r') as f:
        linkedin_data = json.load(f)
except FileNotFoundError:
    print(f"Error: The file {file_path} was not found.")
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from the file {file_path}.")
except Exception as e:
    print(f"An error occurred: {e}")

JSON file read successfully.


In [10]:
linkedin_urls = [item.get('LinkedIn URL') for item in linkedin_data if item.get('LinkedIn URL')]
linkedin_ids = [url.split('/in/')[-1].split('/')[0] for url in linkedin_urls]

print("LinkedIn IDs extracted:")
print(linkedin_ids)

LinkedIn IDs extracted:
['vinicius-natal-799a5b140', 'juliayahagi', 'renan-t-mondini-072701377', 'matheus-henrique-ferreira-de-oliveira-44a31a194', 'paulo-izaque', 'fernanda-samecima', 'alexandre-dos-santos-oliveira-3bb3b6208', 'victoria-ramos-4295892a3', 'luiz-henrique-da-silva-546b041b0', 'guilherme-porcel-a923b232a', 'pedro-lucas-godoy', 'isadora-campos-trindade-a2881a297', 'emanuele-queiros-165826237', 'ana-tereza-mazuco-moraes-677638287', 'amanda-corr%C3%AAa-tavares-53b2532a9', 'amandaramoslt', 'caio-r-de-oliveira-01', 'enzo-guzzo-duz-b44168325', 'francisco-alcantara-b2a0a6209', 'franciscocatoira', 'gabriel-tjustino', 'giovannagarbelotti', 'giovannidionizio', 'giuliagarciaolv', 'guilherme-portilho-753793265', 'gustavo-calonego', 'henrique-reiss', 'isadora-andrade-barbosa', 'jo%C3%A3o-vitor-souza-da-silva', 'j%C3%B4natas-pereira-b99b87324', 'jos%C3%A9-fernando-freire', 'juan-marco-andrade-van-melis-b5a3871b9', 'juliabgloor', 'j%C3%BAlia-dias-895b321b9', 'leonardo-vaz-moreira-097a72

In [12]:
import requests
import json
from linkdapi import LinkdAPI
import time

# Initialize with your API key
api_key = "li-r9CRYTKjdUbULQsdXtvk39_MmzxlmKzkK7I5zgouC42XMG9ZZcdx1KDhNwVS88iUSiGB4eCqiAt6hDgQpPQvcyD0zUNpbQ"
headers = {"X-linkdapi-apikey": f"{api_key}"}

all_profiles_data = []
not_found_profile = []

for profile_id in linkedin_ids:
  combined_data = {}

  # profile overview
  url_ov = f"https://linkdapi.com/api/v1/profile/overview?username={profile_id}"
  overview_response = requests.get(url_ov, headers=headers)

  if overview_response.status_code == 200:
      overview_data = overview_response.json()
      if overview_data and overview_data.get('success'):
          combined_data.update(overview_data.get('data', {}))
          urn = overview_data.get('data', {}).get('urn')

          if urn:
              # profile skills
              url_skills = f"https://linkdapi.com/api/v1/profile/skills?urn={urn}"
              skills_response = requests.get(url_skills, headers=headers)
              if skills_response.status_code == 200:
                  skills_data = skills_response.json()
                  if skills_data and skills_data.get('success'):
                      combined_data['skills'] = skills_data.get('data', {}).get('skills', [])
              else:
                  print(f"Error fetching skills for {profile_id}: Status Code {skills_response.status_code}")


              # profile certifications
              url_cert = f"https://linkdapi.com/api/v1/profile/certifications?urn={urn}"
              certifications_response = requests.get(url_cert, headers=headers)
              if certifications_response.status_code == 200:
                  certifications_data = certifications_response.json()
                  if certifications_data and certifications_data.get('success'):
                      combined_data['certifications'] = certifications_data.get('data', {}).get('certifications', [])
              else:
                  print(f"Error fetching certifications for {profile_id}: Status Code {certifications_response.status_code}")


              # profile education
              url_educ = f"https://linkdapi.com/api/v1/profile/education?urn={urn}"
              education_response = requests.get(url_educ, headers=headers)
              if education_response.status_code == 200:
                  education_data = education_response.json()
                  if education_data and education_data.get('success'):
                      combined_data['education'] = education_data.get('data', {}).get('education', [])
              else:
                  print(f"Error fetching education for {profile_id}: Status Code {education_response.status_code}")

      else:
        not_found_profile.append(profile_id)
        print(f"Error fetching overview for {profile_id}: {overview_data.get('message', 'Unknown Error')}")
  else:
      not_found_profile.append(profile_id)
      print(f"Error fetching overview for {profile_id}: Status Code {overview_response.status_code}")

  all_profiles_data.append(combined_data)
  time.sleep(1) # Add a small delay to avoid hitting the rate limit

# Convert the combined list of dictionaries to a JSON string
combined_json = json.dumps(all_profiles_data, indent=4)

print(combined_json)

Error fetching education for juliayahagi: Status Code 429
Error fetching overview for renan-t-mondini-072701377: Status Code 429
Error fetching overview for matheus-henrique-ferreira-de-oliveira-44a31a194: Status Code 429
Error fetching overview for paulo-izaque: Status Code 429
Error fetching overview for fernanda-samecima: Status Code 429
Error fetching overview for alexandre-dos-santos-oliveira-3bb3b6208: Status Code 429


KeyboardInterrupt: 

In [ ]:
output_file_path = 'combined_linkedin_data.json'

try:
    with open(output_file_path, 'w') as f:
        f.write(combined_json)
    print(f"Successfully saved combined data to {output_file_path}")
except IOError as e:
    print(f"Error writing file {output_file_path}: {e}")